## 데이터 수집
### 키워드 목록 
* KCDC, UN, WHO 선정 감염병 

![Keyword](https://user-images.githubusercontent.com/74284500/98910472-8cf79780-2506-11eb-87ea-1a34e7904936.png)


In [ ]:
import requests
import datetime
from dateutil.relativedelta import relativedelta
import re
import psycopg2 as pg2
from TermRanking import TermManager, TermExtractor
import Collection
import nltk


In [ ]:
EXCEPTION_PATH 
DATA_PATH 
API_KEY 


### 날짜별 data size check

In [ ]:
def getLength(nation, date):
    year = date[0:4]
    month = date[4:6]
    day = date[6:8]

    url = ('https://newsapi.org/v2/everything?'
           'q='+nation+' AND (' + Collection.getDiseaseList() + ')&'
           'from=' + year + '-' + month + '-' + day + 'T00:00:00&'
           'to=' + year + '-' + month + '-' + day + 'T23:59:59&'
           'pageSize=100&'
           'apiKey=' + API_KEY)
    response = requests.get(url)
    totalResult = response.json()['totalResults']
    return (totalResult//100)+1

### Data save funtion

In [ ]:
#데이터 저장함수
def saveData(filePath):
    nation = str(filePath).split('/')[2].split('_')[0]
    date = str(filePath).split('/')[2].split('_')[1]
    year = date[0:4]
    month = date[4:6]
    day = date[6:8]

    length = getLength(nation, date)

    descriptions = []
    urls = []
    for page in range(1,length+1):
        url = ('https://newsapi.org/v2/everything?'
            'q=' + nation + ' AND (' + Collection.getDiseaseList() + ')&'
            'from=' + year + '-' + month + '-' + day + 'T00:00:00&'
            'to=' + year + '-' + month + '-' + day + 'T23:59:59&'
            'pageSize=100&'
            'page='+ str(page) + '&'
            'apiKey=' + API_KEY)

        response = requests.get(url)
        articles = response.json()['articles']
        for article in range(0, len(articles)):
            description = str(articles[article]['description'])
            if "\n" in description:
                continue
            descriptions.append(str(articles[article]['description']))
            urls.append(str(articles[article]['url']))

    saveFile = open(filePath, 'w', encoding='UTF8')

    for description, url in zip(descriptions, urls):
        saveFile.writelines(date + '\t' + description + '\t' + url + '\n')

    saveFile.close()
    print('------- Save Data [ ' + date + ' , ' + nation + ' ] -------')

def mediacontentCreate(nation, date):

    filePath = DATA_PATH + nation + '_' + date + "_" + "mediacontents.txt"

    saveData(filePath)


    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    sql = 'insert into nation_mediacontents_tbl (continent, nation, date, content, url) values '

    file = open(filePath, 'r', encoding='UTF8')
    # nation = str(filePath).split('/')[2].split('_')[0]
    continent = Collection.getContinent(nation)

    lines = file.readlines()

    if not lines:
        return

    for line in lines:
        try:
            elements = line.split('\t')
            date = elements[0]
            content = elements[1]
            url = elements[2]
        except:
            continent = ''
            url = ''

        if '\'' in content:
            content = content.replace('\'', '\'\'')

        sql += "('" + continent + "', '" + nation + "', '" + date + "', '" + content + "', '" + url + "'),"

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    file.close()
    cur.close()
    conn.close()

    print('------- Mediacontents [ ' + date + ' , ' + nation + ' ] -------')

def termsCreate(nation, date):
    host = 'epidemic.co.kr'
    dbname = 'BJang'
    user = 'postgres'
    password = 'Jang1234'
    port = '54321'

    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    sql = "insert into nation_terms_tbl values "

    filePath = DATA_PATH + nation + '_' + date + "_" + "mediacontents.txt"

    file = open(filePath, 'r', encoding='UTF8')
    # nation = str(filePath).split('/')[2].split('_')[0]
    continent = Collection.getContinent(nation)
    # date = str(filePath).split('/')[2].split('_')[1]

    lines = file.readlines()

    if not lines:
        return

    contents = []

    for line in lines:
        try:
            content = line.split('\t')[1]
            contents.append(content)
        except:
            continue

    termManager = TermManager.create(None, None, None, contents)

    for content in contents:
        termList = TermExtractor.create(None, content).termList
        termManager.addTermObject(termList=termList)
    termManager.calculate()

    termList = termManager.termObjectList

    for termObject in termList:
        if '\'' in termObject.term:
            termObject.term = termObject.term.replace('\'', '\'\'')

        sql += (
            "('" + date + "', '" + continent + "', '" + nation + "', '" + termObject.term + "', " + str(termObject.count) + ", " + str(termObject.countByContents)
            + ", " + str(termObject.tf) + ", " + str(termObject.tfLog) + ", " + str(termObject.idf) + ", " + str(termObject.pIdf) + ", " + str(termObject.tfidf) + ", " + str(termObject.tfidfWithLog)
            + ", " + str(termObject.RSJweight) + ", " + str(termObject.nCosineWithTF) + ", " + str(termObject.nCosineWithTFLog) + ", " + str(termObject.nMaximumTF) + ", " + str(termObject.smart)
            + ", " + str(termObject.inquery) + ", " + str(termObject.pCosine) + ", " + str(termObject.pByte) + ", " + str(termObject.pUnique) + ", " + str(termObject.aidf) + ", " + str(termObject.BM25)
            + ", " + str(termObject.nBM25) + ", " + str(termObject.qBM25) + ", " + str(termObject.TREC8) + ", " + str(termObject.WBR99) + "),"
        )

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    file.close()
    cur.close()
    conn.close()

    print('------- Terms [ ' + date + ' , ' + nation + ' ] -------')

def regionCreate(nation):
    host = 'epidemic.co.kr'
    dbname = 'BJang'
    user = 'postgres'
    password = 'Jang1234'
    port = '54321'

    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    sql = "insert into nation_region_tbl values "

    continent = Collection.getContinent(nation)
    regions = Collection.getRegionList(nation)

    one_month_ago = datetime.date.today() - relativedelta(months=1)
    year = one_month_ago.strftime('%Y')
    month = one_month_ago.strftime('%m')
    date = year + month

    for regions in regions:
        url = ('https://newsapi.org/v2/everything?'
               'q=' + nation + ' AND (' + Collection.getDiseaseList() + ') AND ' + regions + '&'
               'from=' + year + '-' + month + '-' + '01' + 'T00:00:00&'
               'to=' + year + '-' + month + '-' + '31' + 'T23:59:59&'
               'pageSize=100&'
               'apiKey=' + API_KEY)
        response = requests.get(url)
        totalResult = response.json()['totalResults']

        sql += (
            "('" + date + "', '" + continent + "', '" + nation + "', '" + regions + "', " + str(totalResult) + "),"
        )

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    cur.close()
    conn.close()

    print('------- Regions [ ' + date + ' , ' + nation + ' ] -------')

def countCreate(nation):
    host = 'epidemic.co.kr'
    dbname = 'BJang'
    user = 'postgres'
    password = 'Jang1234'
    port = '54321'

    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    one_month_ago = datetime.date.today() - relativedelta(months=1)
    year = one_month_ago.strftime('%Y')
    month = one_month_ago.strftime('%m')
    date = year + month

    sql = (
        "select * from nation_mediacontents_tbl "
        "where"
        "   date >= '" + date + "01'" + " and date <= '" + date + "31' and nation = '" + nation + "'"
    )

    cur.execute(sql)
    rows = cur.fetchall()

    diseaseTerms = re.sub("[()]", "", Collection.getDiseaseList())
    diseaseTerms = diseaseTerms.replace(" AND ", " ")
    diseaseTerms = diseaseTerms.split(" OR ")

    diseaseObjectList = []

    for diseaseTerm in diseaseTerms:
        diseaseObjectList.append({"term": diseaseTerm, "count": 0})

    for row in rows:
        article = str(row[4])
        for diseaseObject in diseaseObjectList:
            if diseaseObject['term'].lower() in article.lower():
                diseaseObject['count'] += 1

    continent = Collection.getContinent(nation)

    sql = "insert into nation_count_tbl values "

    for diseaseObject in diseaseObjectList:
        sql += "('" + date + "', '" + continent + "', '" + nation + "', '" + diseaseObject['term'] + "', " + str(diseaseObject['count']) + "),"

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    cur.close()
    conn.close()

    print('------- Count [ ' + date + ' , ' + nation + ' ] -------')

In [ ]:
if __name__ == '__main__':

    try:
        today = datetime.date.today()
        yesterDay = datetime.date.today() - datetime.timedelta(1)
        date = yesterDay.strftime('%Y%m%d')

        nations = Collection.getNationList()

        for nation in nations:
            mediacontentCreate(nation, date)
            termsCreate(nation, date)

    except Exception as e:
        time = datetime.datetime.now()
        exceptionFile = open(EXCEPTION_PATH + today.strftime('%Y%m%d') + '_' + 'exception.txt', 'a', encoding='utf8')
        exceptionFile.write(str(time) + '\t' + e.__str__() + '\n')

